# Extracting Resonance Parameter Covariances from an ENDF file

The covariance file has the same hierarchy as MF2: Section, Isotope, Energy range

## Reich-Moore with General Resolved Resonance Formats (LCOMP = 1)

In [ ]:
import ENDFtk

Pb208_tape = ENDFtk.tree.Tape.from_file( 'resources/n-Pb208.endf' )
Pb208_mat = Pb208_tape.materials.front()

Pb208_file32 = Pb208_mat.file(32).parse()
Pb208_section151 = Pb208_file32.section(151)
Pb208_resonance_region = Pb208_section151.isotopes[0]

Pb208_resonance_region

In [ ]:
print(f"ZA of this isotope (ZAI): {Pb208_resonance_region.ZAI}")
print(f"number of resonance ranges (NER): {Pb208_resonance_region.number_resonance_ranges}")

The RRR is the first range

In [ ]:
Pb208_RRR = Pb208_resonance_region.resonance_ranges[0]
Pb208_RRR

In [ ]:
print(f"Resonance range limits: {Pb208_RRR.lower_energy:.1E} - {Pb208_RRR.upper_energy:.1E} eV")
print(f"Type (LRU): {Pb208_RRR.type}")
print(f"Representation (LRF): {Pb208_RRR.representation}")
print(f"Energy-dependent scattering radius: {Pb208_RRR.energy_dependent_scattering_radius}")
print(f"Scattering radius calculation method (NAPS): {Pb208_RRR.scattering_radius_calculation_option}")

The parameters and covariance matrices are in the `parameters` attribute

In [ ]:
Pb208_parameters = Pb208_RRR.parameters
Pb208_parameters

In [ ]:
print(f"Target spin (SPI): {Pb208_parameters.spin}")
print(f"RMatrix formalism (LRF): {Pb208_parameters.representation}")
print(f"Covariance representation (LCOMP): {Pb208_parameters.covariance_representation}")
print(f"Scattering radius (AP): {Pb208_parameters.scattering_radius}")
print(f"There is scattering radius uncertainty (ISR): {Pb208_parameters.scattering_radius_uncertainty_flag}")
print(f"Number of short-range covariance blocks (NSRS): {Pb208_parameters.number_short_range_blocks}")
print(f"Number of long-range covariance blocks (NLRS): {Pb208_parameters.number_long_range_blocks}")

If there is scattering radius uncertainty, it is in the `scattering_radius_uncertainty` attribute

In [ ]:
Pb208_radius_unc = Pb208_parameters.scattering_radius_uncertainty
Pb208_radius_unc

In [ ]:
print(f"The default radius uncertainty (DAP): {Pb208_radius_unc.default_uncertainty}")
print(f"The l-dependent radius uncertainties (DAPL): {Pb208_radius_unc.uncertainties[:]}")

If there are short-range covariance blocks, the format is dependent on the RMatrix formalism used. For Reich-Moore, the parameters from File 2 are repeated along with the covariance matrix as an upper-triangular matrix.

In [ ]:
Pb208_short_range = Pb208_parameters.short_range_blocks[0]
Pb208_short_range

In [ ]:
print(f"Number of resonances (NRB): {Pb208_short_range.number_resonances}")
print(f"For each resonances, number of parameters with covariances (MPAR): {Pb208_short_range.number_parameters_with_covariances}")
print(f"Covariance matrix order (NPARB): {Pb208_short_range.covariance_matrix_order}   (which equals NRB*MPAR)")
print(f"Number of values in the upper-triangular matrix (NVS): {Pb208_short_range.number_values}   (which equals NPARB * (NPARB + 1) / 2)")

The parameters (for all spin groups) are repeated:

In [ ]:
print(f"Energy [eV] \t J \t Gamma_n \t Gamma_g \t Gamma_f1 \t Gamma_f2")
print("-"*82)
for Er, J, n, g, f1, f2 in zip(Pb208_short_range.resonance_energies,
                               Pb208_short_range.spin_values, 
                               Pb208_short_range.neutron_widths, 
                               Pb208_short_range.gamma_widths, 
                               Pb208_short_range.first_fission_widths, 
                               Pb208_short_range.second_fission_widths):
    print(f"{Er:.2E} \t {J} \t {n}    \t {g}    \t {f1}   \t         {f2}")

And the covariance matrix is presented as an upper-triangular matrix in list form

In [ ]:
Pb208_short_range.covariance_matrix[:]

The list can be converted to a full symmetric matrix using numpy functions:

In [ ]:
import numpy as np

def fill_in_matrix(upper_triangular_list, matrix_order):

    # check that the inputs are consistent
    if not len(upper_triangular_list) == matrix_order * (matrix_order + 1) / 2:
        raise ValueError(f"The length of the matrix elements is not consistent with the stated matrix order")

    # create empty matrix
    covariance_matrix = np.zeros((matrix_order, matrix_order))

    # get the indices of the upper triangular values
    indices = np.triu_indices(matrix_order)

    # fill in the matrix
    covariance_matrix[indices] = upper_triangular_list

    # fill in the lower triangle
    covariance_matrix += np.triu(covariance_matrix,k=1).T

    return covariance_matrix

cov = fill_in_matrix(Pb208_short_range.covariance_matrix, Pb208_short_range.covariance_matrix_order)

print(cov[:5,:5])